In [1]:
from bs4 import BeautifulSoup
import datetime
import requests
i = 1

def get_soup(i):
    soup = None
    try:
        page = requests.get("https://www.tiebreak.co.uk/events/search/539253e8756391661f9e033a6dacdd2d/{}".format(i))
        soup = BeautifulSoup(page.content, 'html.parser')
    except:
        pass
    return soup

def get_soup_url(url):
    soup = None
    try:
        url = "https:{}".format(url)
        print(url)
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
    except:
        pass
    return soup


def get_soup_standings(url):
    soup = None
    try:
        url = "https:{}".format(url+'/standings')
        print(url)
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
    except:
        pass
    return soup    

def get_date(i, year):
    i = i.contents[0].replace('st ',' ').replace('nd ', ' ').replace('rd ', ' ').replace('th ', ' ')
    return datetime.datetime.strptime('{} {}'.format(year, i), '%Y %A %d %b at %I:%M%p')


def get_soup_games(url):
    soup = None
    try:
        url = "https:{}".format(url+'/feed')
        print(url)
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
    except:
        pass
    return soup   


def get_soup_url_round(url, i):
    soup = None
    try:
        url = "https:{}".format(url+'/round/round-{}'.format(i))
        print(url)
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
    except:
        pass
    return soup

def get_round_result(soup, date, hours):
    result = []
    clean_result = []
    final_result = []
    try:
        top = soup.find_all(class_='pairing-top')
        for i in top:
            result.append([j.contents for j in i.find_all('td')])
        bottom = soup.find_all(class_='pairing-bottom')
        for i in bottom:
            result.append([j.contents for j in i.find_all('td')])
        for i in result:
            table = int(i[0][0])
            name = i[1][0].strip() if len(i[1]) == 2 else i[1][0].contents[0].strip()
            guild = i[1][1].contents[0].strip()[1:-1] if len(i[1]) == 2 else i[1][2].contents[0].strip()[1:-1]
            home = int(i[2][0] if len(i[1]) == 2 else i[2][0].contents[0])
            away = int(i[3][0].contents[0] if len(i[1]) == 2 else i[3][0])
            entry = [table, name, guild, home, away]
            clean_result.append(entry)
        clean_result = sorted(clean_result, key = lambda i: (i[0], -i[3]))
        if not isinstance(clean_result[0][0], int):
            print(clean_result)
        for i in range(int(len(clean_result)/2)):
            one = clean_result[i*2]
            two = clean_result[i*2+1]
            final_result.append([date+datetime.timedelta(seconds = 3600*(11+j)), one[1], one[2], two[1], two[2], one[3], one[4], one[3]==one[4]])
    except:
        pass
    return final_result


data= []
l=0
soup = get_soup(i)
while soup.find_all(class_="events_listing"):
    for event in soup.find_all(class_="events_listing"):
        url = event.find_all(class_='name')[0].a['href']
        date = event.find_all(class_='date')[0].find_all('span')[0].contents[0].replace('\t', '').replace('\n', '')
        if '-' in date:
            date = date.split('-')[1]
        date = date.replace('st','').replace('nd','').replace('rd','').replace('th','')
        date = datetime.datetime.strptime(date, '%d %b %Y')
        print(url)
        print(date)
        j=1
        soup2 =  get_soup_standings(url)
        try:
            players = {a.find_all('a')[0].contents[0]:(b.contents[0] if b.contents else None) for a,b  in zip(*[iter(soup2.find_all(class_='standings-table')[0].find_all(class_='name')[2:])]*2)}
            soup2 = get_soup_games(url)
            dates = [get_date(i, date.year) for i in soup2.find_all(class_='ts')]
            pairings = [i.a.contents[0].split(' beat ') for i in soup2.find_all(class_='winner')]
            scores = [i.contents[0].replace('\n','').replace('F','').replace('A', '').replace('  ',' ').split(':')[1].strip().split(' ')[:2] for i in soup2.find_all(class_='meta') if ':' in i.contents[0]]
            result = [[c, a[0], players[a[0]], a[1], players[a[1]]]+b+[False] for a, b, c in zip(pairings, scores, dates) if (len(a) > 1) and a[0] in players and a[1] in players]
        except:
            print('Not working: {}'.format(url))
        data.extend(result)
    l += 1
    soup = get_soup(l)




In [40]:
import pickle
data = [[i[0],i[1],str(i[2]),i[3],str(i[4]),i[5],i[6],i[7]] for i in data if len(i) == 8]
pickle.dump( data, open( "data.pcl", "wb" ) )

In [2]:

# data= []
# i=0
# soup = get_soup(i)
# while soup:
#     for event in soup.find_all(class_="events_listing"):
#         url = event.find_all(class_='name')[0].a['href']
#         date = event.find_all(class_='date')[0].find_all('span')[0].contents[0].replace('\t', '').replace('\n', '')
#         if '-' in date:
#             date = date.split('-')[1]
#         date = date.replace('st','').replace('nd','').replace('rd','').replace('th','')
#         date = datetime.datetime.strptime(date, '%d %b %Y')
#         print(url)
#         print(date)
#         j=1
#         soup2 =  get_soup_url_round(url, j)
#         check = True
#         while soup2 and check:
#             result = get_round_result(soup2, date, j)
#             if not result:
#                 check = False
#             data.extend(result)
#             j+=1
#             soup2 =  get_soup_url_round(url, j)
#     i += 1
#     soup = get_soup(i)

